In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local[6]") \
        .appName('Stream w/ Pyspark') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/08 23:56:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/08 23:56:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from config import DATADIR
df = spark.read.parquet(DATADIR + 'parquet/stream_df.parquet')

In [3]:
df.createOrReplaceTempView('stream')

22/08/08 23:56:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
import time
from config import RUNTIME, SERVER_PORT, SOURCE_TOPIC
try:
    start_time = time.time()
    last_run = 0
    while last_run <= RUNTIME:
        now = time.time() - start_time
        result = spark.sql(f'SELECT * FROM stream WHERE Stream_Time BETWEEN {last_run} AND {now}')
        last_run = now

        if not result.isEmpty():
                    result \
                    .selectExpr("CAST(ID AS STRING)", "to_json(struct(*)) AS value") \
                    .write \
                    .format('kafka') \
                    .option('kafka.bootstrap.servers',SERVER_PORT) \
                    .option('topic',SOURCE_TOPIC) \
                    .save()
        print(last_run)

except KeyboardInterrupt:
    pass

# The exception below is raised by spark because spark streaming scripts need to be 
# run from terminal

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        

To run this script we have to download it to .py, clean it a bit, then run
`spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 spark_stream.py`.

You will have to find the right package for your spark version.

The `print(last_run)` will be used to test the latency from faust and mongodb. Will be removed from the final version.